### imports

In [1]:
import csv
import psycopg2
import os
import glob
from api_keys import postgres
UNIQUES=set('emp_title_id')

### functions

In [2]:
def load_data_sql(table,columns,source):
    source=os.path.abspath(source)
    combined_columns=', '.join(columns)
    sql=f'''
copy public.{table} ({combined_columns}) 
FROM '{source}' 
DELIMITER ',' 
CSV HEADER 
QUOTE '\"' 
;
    '''
    return sql

def cleanse(astr):
    return astr.encode('ascii', 'replace').decode('ascii').replace('?','')

### create database

psql -U postgres
drop database xxx with (force);
\l
create database xxx;

### create tables and load csv data

In [3]:
print("Connecting to Database")
with psycopg2.connect(f"host=localhost dbname=db_challenge_09 user=postgres password={postgres}") as conn:
    print("Connected")
    csvpath="..\\data\\"
    # Loop through each CSV
    with conn.cursor() as cur:
        for filename in glob.glob(csvpath+"*.csv"):
            # Create a table name
            filename=os.path.normpath(filename)
#             print(filename)
            tablename=filename.replace(csvpath,"").replace(".csv", "")
            tablename=tablename.strip('"')
            print(tablename)
            # Open file
            fileInput=open(filename, "r")
            # Extract first line of file
            firstLine=fileInput.readline().strip()
            # Split columns into an array [...]
            columns=firstLine.split(",")
            columns=[x.strip('"') for x in columns]
            columns=[cleanse(x) for x in columns]
            # Build SQL code to drop table if exists and create table
            sqlQueryCreate='DROP TABLE IF EXISTS '+tablename+" CASCADE;\n"
            sqlQueryCreate+='CREATE TABLE '+tablename+"\n("
            #some loop or function according to your requiremennt
            # Define columns for table
            for column in columns:
                if column.endswith('_no'):
#                     print('no')
#                     sqlQueryCreate+=column+" INT,\n"
                    sqlQueryCreate+=column+" CHAR(10) NOT NULL,\n"
                elif column in ['salary']:
                    sqlQueryCreate+=column+" INT NOT NULL,\n"
                elif column.endswith('date'):
                    sqlQueryCreate+=column+" DATE NOT NULL,\n"
                elif column in UNIQUES:
                    sqlQueryCreate+=column+" VARCHAR(64) NOT NULL UNIQUE,\n"
                else:
                    sqlQueryCreate+=column+" VARCHAR(64) NOT NULL,\n"
            sqlQueryCreate=sqlQueryCreate[:-2]
            sqlQueryCreate+="\n);"
#             print(sqlQueryCreate)
#             print('')
            cur.execute(sqlQueryCreate)
            conn.commit()
            
            #load data
            sqldata=load_data_sql(tablename,columns,filename)
#             print(sqldata)
            cur.execute(sqldata)
            conn.commit()
            

Connecting to Database
Connected
departments
dept_emp
dept_manager
employees
salaries
titles


### create primary keys

In [4]:
primary_keys=[
    'ALTER TABLE public.departments ADD PRIMARY KEY (dept_no);',
    'ALTER TABLE public.dept_emp ADD PRIMARY KEY (emp_no,dept_no);',
    'ALTER TABLE public.dept_manager ADD PRIMARY KEY (emp_no);',
    'ALTER TABLE public.employees ADD PRIMARY KEY (emp_no);',
    'ALTER TABLE public.salaries ADD PRIMARY KEY (emp_no);',
    'ALTER TABLE public.titles ADD PRIMARY KEY (title_id);',
    
#     'ALTER TABLE public.dept_emp ADD PRIMARY KEY (dept_no);',
#     'ALTER TABLE public.dept_manager ADD PRIMARY KEY (emp_no,dept_no);',
]
print("Connecting to Database")
with psycopg2.connect(f"host=localhost dbname=db_challenge_09 user=postgres password={postgres}") as conn:
    print("Connected")
    with conn.cursor() as cur:
        for pk in primary_keys:
            print('attempting: ',pk)
            try:
                cur.execute(pk)
                conn.commit()
                print('committed')
            except Exception as e:
                print('failed!')
                print(e)
                conn.rollback()
        
    

Connecting to Database
Connected
attempting:  ALTER TABLE public.departments ADD PRIMARY KEY (dept_no);
committed
attempting:  ALTER TABLE public.dept_emp ADD PRIMARY KEY (emp_no,dept_no);
committed
attempting:  ALTER TABLE public.dept_manager ADD PRIMARY KEY (emp_no);
committed
attempting:  ALTER TABLE public.employees ADD PRIMARY KEY (emp_no);
committed
attempting:  ALTER TABLE public.salaries ADD PRIMARY KEY (emp_no);
committed
attempting:  ALTER TABLE public.titles ADD PRIMARY KEY (title_id);
committed


### add indexes

In [5]:
additional=[
    'create index salaries_emp_no_idx on salaries(emp_no);',
    'create index dept_emp_emp_no_idx on dept_emp(emp_no);',
    'create index dept_emp_dept_no_idx on dept_emp(dept_no);',
]
print("Connecting to Database")
with psycopg2.connect(f"host=localhost dbname=db_challenge_09 user=postgres password={postgres}") as conn:
    print("Connected")
    with conn.cursor() as cur:
        for add in additional:
            print('attempting: ',add)
            try:
                cur.execute(add)
                conn.commit()
                print('committed')
            except Exception as e:
                print('failed!')
                print(e)
                conn.rollback()

Connecting to Database
Connected
attempting:  create index salaries_emp_no_idx on salaries(emp_no);
committed
attempting:  create index dept_emp_emp_no_idx on dept_emp(emp_no);
committed
attempting:  create index dept_emp_dept_no_idx on dept_emp(dept_no);
committed


### add foreign keys

In [6]:
additional=[
    'alter table salaries add constraint salaries_emp_no foreign key (emp_no) references employees(emp_no);',
    'alter table dept_emp add constraint dept_emp_emp_no foreign key (emp_no) references employees(emp_no);',
#     'alter table departments add constraint departments_dept_no_key unique (dept_no);',
#     'alter table departments add constraint departments_dept_no foreign key (dept_no) references dept_emp(dept_no);',
    'alter table dept_manager add constraint dept_manager_emp_no foreign key (emp_no) references dept_emp(emp_no);',

    
#     'ALTER TABLE public.titles ADD CONSTRAINT constraint_fk FOREIGN KEY (title_id) REFERENCES employees(emp_title_id);',
#     'ALTER TABLE public.dept_emp ADD PRIMARY KEY (emp_no,dept_no);',
#     'ALTER TABLE public.dept_manager ADD PRIMARY KEY (dept_no,emp_no);',
#     'ALTER TABLE public.employees ADD PRIMARY KEY (emp_no);',
#     'ALTER TABLE public.salaries ADD PRIMARY KEY (emp_no);',
#     'ALTER TABLE public.titles ADD PRIMARY KEY (title_id);',
]
print("Connecting to Database")
with psycopg2.connect(f"host=localhost dbname=db_challenge_09 user=postgres password={postgres}") as conn:
    print("Connected")
    with conn.cursor() as cur:
        for add in additional:
            print('attempting: ',add)
            try:
                cur.execute(add)
                conn.commit()
                print('committed')
            except Exception as e:
                print('failed!')
                print(e)
                conn.rollback()

Connecting to Database
Connected
attempting:  alter table salaries add constraint salaries_emp_no foreign key (emp_no) references employees(emp_no);
committed
attempting:  alter table dept_emp add constraint dept_emp_emp_no foreign key (emp_no) references employees(emp_no);
committed
attempting:  alter table dept_manager add constraint dept_manager_emp_no foreign key (emp_no) references dept_emp(emp_no);
failed!
there is no unique constraint matching given keys for referenced table "dept_emp"

